In [4]:
import numpy as np
import matplotlib.pyplot as plt

from agent_distribution import AgentDistribution
from utils import compute_continuity_noise, compute_contraction_noise

In [5]:
def policy_loss(agent_dist, beta, s, sigma, true_beta=None):
    """Method that returns the policy loss incurred given an agent distribution and model and threshold.
    Assumes that there is an model true_beta when applied to the agents' hidden eta features 
    optimally selects the top agents.
    
    Keyword args:
    agent_dist -- AgentDistribution
    beta -- model parameters (N, 1) array
    s -- threshold (float)
    sigma -- standard deviation of the noise (float)
    true_beta -- (N, 1) array
    """
    if true_beta == None:
        true_beta = np.zeros(beta.shape)
        true_beta[0] = 1.
  
    true_agent_types = agent_dist.n_agent_types
    noisy_scores = agent_dist.best_response_noisy_score_distribution(beta, s, sigma)
    positive_allocation = (noisy_scores >= s).flatten()

    etas = agent_dist.get_etas()
    etas_for_positive_allocation = etas[positive_allocation]
    
    positive_allocation_mean_true_score = np.mean([np.matmul(true_beta.T, eta).item() for eta in etas_for_positive_allocation]).item()
    return -positive_allocation_mean_true_score
    
    
def find_optimal_beta(agent_dist, sigma, q, plot=False):
    """Method that returns the optimal model parameters given an agent distribution.
    
    Method assumes that data is two dimensional (because model parameters are assumed to lie on unit circle.)
    
    Keyword args:
    agent_dist -- AgentDistribution
    q -- quantile of interest
    
    Returns:
    opt_beta -- optimal model (N, 1) array
    opt_s_beta -- fixed point corresponding to optimal model (float)
    loss -- loss incurred at optimal beta (float)
    """
    dim = agent_dist.d
    assert dim==2, "Method does not work for dimension {}".format(dim)
    
    thetas = np.linspace(0, 2 * np.pi, 50)
    fixed_points = []
    betas = []
    
    #compute beta and fixed point for each theta
    print("Computing fixed points...")
    for theta in thetas:
        beta = np.array([np.sin(theta), np.cos(theta)]).reshape(dim, 1)
        fp = agent_dist.quantile_fixed_point_polyfit(beta, sigma, q)
        fixed_points.append(fp)
        betas.append(beta)
    
    if plot:
        plt.plot(thetas, fixed_points)
        plt.xlabel("Thetas (corresponds to different Beta)")
        plt.ylabel("s_beta")
        plt.title("Location of Fixed Points: s_beta vs. beta")
        plt.show()
        plt.close()
        
    #compute loss at each point, given a particular beta
    print("Computing losses...")
    losses = []
    for i in range(len(thetas)):
        losses.append(policy_loss(agent_dist, betas[i], fixed_points[i], sigma))
        
    if plot:
        plt.plot(thetas, losses)
        plt.xlabel("Theta (corresponds to different beta)")
        plt.ylabel("Loss")
        plt.title("Loss at (beta, s_beta)")
        plt.show()
        plt.close()
        
    idx = np.argmin(losses)
    loss = losses[idx]
    opt_beta = betas[idx]
    opt_s_beta = fixed_points[idx]
    return loss, opt_beta, opt_s_beta
        
    
    
    
    
        
        
        
    
    

In [6]:
agent_dist = AgentDistribution(n=10000, n_types=100)
sigma_continuity = compute_continuity_noise(agent_dist)
sigma_contraction = compute_contraction_noise(agent_dist)
q = 0.7


In [ ]:
find_optimal_beta(agent_dist, sigma_continuity, q, plot=True)

Computing fixed points...


In [ ]:
find_optimal_beta(agent_dist, sigma_contraction, q, plot=True)